In [1]:
from selenium import webdriver
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
my_password = getpass()

········


In [8]:
driver = webdriver.Firefox()
driver.get('http://www.twitter.com/login')

In [9]:
username = driver.find_element_by_xpath('//input[@name = "username"]')
username.send_keys('mathjkim')
username.send_keys(Keys.RETURN)

<ipython-input-9-3850a1cd7f64>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath('//input[@name = "username"]')


In [10]:
password = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/div/label/div/div[2]/div/input')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

<ipython-input-10-15012e0ee9f3>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/div/label/div/div[2]/div/input')


In [11]:
search_input = driver.find_element_by_xpath('//input[@placeholder="Search Twitter"]')
search_input.send_keys('#wallstreetbets')
search_input.send_keys(Keys.RETURN)

<ipython-input-11-09bf0f2dd708>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_input = driver.find_element_by_xpath('//input[@placeholder="Search Twitter"]')


In [12]:
driver.find_element_by_link_text('Latest').click()

<ipython-input-12-7f8c586bf963>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('Latest').click()


In [19]:
def get_tweet_data(card):
    user_date = card.find_element_by_xpath('./div[1]/div[1]/div[1]/div[2]/div[2]/div[1]')
    username = user_date.text
    try:
        postdate = user_date.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    postcontent = card.find_element_by_xpath('./div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
    reply_count = card.find_element_by_xpath('.//div[@data-testid = "reply"]').text
    retweet_count = card.find_element_by_xpath('.//div[@data-testid = "retweet"]').text
    like_count = card.find_element_by_xpath('.//div[@data-testid = "like"]').text
    
    tweet = (username, postdate, postcontent, reply_count, retweet_count, like_count)
    return tweet

In [20]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid = "tweet"]')

    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)      
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    scroll_attempt = 0
    while True:
    #check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        current_position = driver.execute_script("return window.pageYOffset;")
        if last_position == current_position:
            scroll_attempt += 1
        
        #end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = current_position
            break

<ipython-input-20-a3773fd42f35>:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  page_cards = driver.find_elements_by_xpath('//article[@data-testid = "tweet"]')
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


StaleElementReferenceException: Message: The element reference of <div class="css-1dbjc4n"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.jsm:442:5
element.resolveElement@chrome://remote/content/marionette/element.js:686:11
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:253:26
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:261:29
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:261:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:79:29


In [21]:
len(data)

624

In [24]:
with open('wallstreetbets_tweets.csv','w', newline = '', encoding = 'utf-8') as f:
    header = ['UserName', 'Timestamp', 'Post', 'Reply', 'Retweet', 'Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)